In [1]:
import pandas as pd

In [1]:
# %pip install tensorflow[and-cuda]

In [2]:
import os

dir_notebook = os.getcwd()
print(dir_notebook)

/home/david/proyectos4/repositorios/Proyectos-SGBA1/Pruebas/Modelos/precio_peninsula


In [3]:
ruta_trabajo = '../../../'

# Cambiar el directorio de trabajo
os.chdir(ruta_trabajo)

print("Directorio de trabajo actual:", os.getcwd())

Directorio de trabajo actual: /home/david/proyectos4/repositorios/Proyectos-SGBA1


In [4]:
!pwd

/home/david/proyectos4/repositorios/Proyectos-SGBA1


### Cargar df_clima_precio 

In [5]:
df_precio = pd.read_parquet('data/processed/datos_precio/clima_precio_merged.parquet')

In [6]:
# !pip install numpy pandas scikit-learn tensorflow xgboost lightgbm matplotlib

In [7]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from tensorflow.keras.layers import LSTM, Dense, Dropout

import matplotlib.pyplot as plt

2025-03-22 20:39:56.612128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742672396.626973   37190 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742672396.630828   37190 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742672396.644129   37190 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742672396.644150   37190 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742672396.644152   37190 computation_placer.cc:177] computation placer alr

In [8]:
df_precio.columns

Index(['€/kwh', 'fecha', 'indicativo_3196', 'altitud_3196', 'tmed_3196',
       'tmin_3196', 'tmax_3196', 'velmedia_3196', 'racha_3196', 'sol_3196',
       ...
       'horaPresMax_cos_8175', 'horaPresMin_varias_8175',
       'horaPresMin_minutos_8175', 'horaPresMin_sin_8175',
       'horaPresMin_cos_8175', 'prec_inapreciable_8175', 'prec_valor_8175',
       'prec_log_8175', 'dir_sin_8175', 'dir_cos_8175'],
      dtype='object', length=479)

In [9]:
# Identify numeric and non-numeric columns
numeric_cols = df_precio.select_dtypes(include=['float64', 'int64']).columns
non_numeric_cols = df_precio.select_dtypes(exclude=['float64', 'int64']).columns

print("Non-numeric columns:", non_numeric_cols)
print(df_precio[non_numeric_cols].head())

# Remove all non-numeric columns except 'fecha'
columns_to_drop = [col for col in non_numeric_cols if col != 'fecha']
df_precio = df_precio.drop(columns_to_drop, axis=1, errors='ignore')
df_precio_coor = df_precio.drop(non_numeric_cols, axis=1, errors='ignore')


Non-numeric columns: Index(['fecha', 'horaHrMax_3196', 'horaHrMin_3196', 'horaHrMax_9434',
       'horaHrMin_9434', 'horaHrMax_9170', 'horaHrMin_9170', 'horaHrMax_2539',
       'horaHrMin_2539', 'indicativo_6155A', 'horaHrMax_6155A',
       'horaHrMin_6155A', 'horaHrMax_1505', 'horaHrMin_1505',
       'indicativo_3469A', 'horaHrMax_3469A', 'horaHrMin_3469A',
       'horaHrMax_8019', 'horaHrMin_8019', 'indicativo_0200E',
       'horaHrMax_4121', 'horaHrMin_4121', 'horaHrMax_8175', 'horaHrMin_8175'],
      dtype='object')
       fecha horaHrMax_3196 horaHrMin_3196 horaHrMax_9434 horaHrMin_9434  \
0 2014-11-03         Varias         Varias         Varias         Varias   
1 2014-11-03         Varias         Varias         Varias         Varias   
2 2014-11-03         Varias         Varias         Varias         Varias   
3 2014-11-03         Varias         Varias         Varias         Varias   
4 2014-11-03         Varias         Varias         Varias         Varias   

  horaHrMax_9170 

In [ ]:
# Calculamos la correlación con €/kwh
correlations = df_precio.corr()['€/kwh'].sort_values(ascending=False)

# Mostramos las 10 variables más correlacionadas (excluyendo la propia €/kwh)
print("Top 10 correlaciones con €/kwh:")
print(correlations[1:11])

# Usamos Random Forest para ver la importancia de las características
# Seleccionamos solo columnas numéricas
numeric_cols = df_precio.select_dtypes(include=['float64', 'int64']).columns
X = df_precio[numeric_cols].drop('€/kwh', axis=1)
y = df_precio['€/kwh']

# Entrenamos un Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)

# Obtenemos la importancia de las características
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
})

# Mostramos las 10 características más importantes según Random Forest
print("\nTop 10 características más importantes según Random Forest:")
print(feature_importance.sort_values('importance', ascending=False).head(10))

# Visualizamos las 10 características más importantes
plt.figure(figsize=(12, 6))
top_10_features = feature_importance.sort_values('importance', ascending=False).head(10)
plt.bar(top_10_features['feature'], top_10_features['importance'])
plt.xticks(rotation=45, ha='right')
plt.title('Top 10 características más importantes para predecir €/kwh')
plt.tight_layout()
plt.show()

In [10]:
# Ordenar por fecha
df_precio = df_precio.sort_values('fecha')

# Seleccionar todas las columnas numéricas excepto la variable objetivo
X = df_precio_coor.drop('€/kwh', axis=1)
y = df_precio['€/kwh']

# Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [11]:
# Select top 20 most important features
# Get temperature, sun and wind related features
selected_features = [col for col in X.columns if any(x in col for x in ['tmed', 'sol', 'velmedia'])]
X_reduced = X_scaled[:, [list(X.columns).index(feat) for feat in selected_features]]

# Create sequences with reduced features
def create_sequences(X, y, seq_length=72, batch_size=1000):
    n_samples = len(X) - seq_length
    X_seq = np.zeros((n_samples, seq_length, X.shape[1]))
    y_seq = np.zeros(n_samples)
    
    for i in range(0, n_samples, batch_size):
        end_idx = min(i + batch_size, n_samples)
        for j in range(i, end_idx):
            X_seq[j] = X[j:j+seq_length]
            y_seq[j] = y[j+seq_length]
            
    return X_seq, y_seq

X_seq, y_seq = create_sequences(X_reduced, y.values)

# Print the shape of the sequences
print("X_seq shape:", X_seq.shape)
print("y_seq shape:", y_seq.shape)

X_seq shape: (57553, 72, 36)
y_seq shape: (57553,)


In [12]:
selected_features

['tmed_3196',
 'velmedia_3196',
 'sol_3196',
 'tmed_9434',
 'velmedia_9434',
 'sol_9434',
 'tmed_9170',
 'velmedia_9170',
 'sol_9170',
 'tmed_2539',
 'velmedia_2539',
 'sol_2539',
 'tmed_6155A',
 'velmedia_6155A',
 'sol_6155A',
 'tmed_5514',
 'sol_5514',
 'velmedia_5514',
 'tmed_1505',
 'velmedia_1505',
 'sol_1505',
 'tmed_3469A',
 'velmedia_3469A',
 'sol_3469A',
 'tmed_8019',
 'velmedia_8019',
 'sol_8019',
 'tmed_0200E',
 'velmedia_0200E',
 'sol_0200E',
 'tmed_4121',
 'velmedia_4121',
 'sol_4121',
 'tmed_8175',
 'velmedia_8175',
 'sol_8175']

In [13]:
%pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [16]:
import optuna
from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input
from sklearn.metrics import mean_squared_error
import numpy as np

# Definir función objetivo para Optuna
def objective(trial):
    # Definir el espacio de búsqueda de hiperparámetros
    lstm_units = trial.suggest_int('lstm_units', 20, 64)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    # Inicializar TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=3)  # You can adjust the number of splits
    
    mse_scores = []
    
    # Loop through each fold in TimeSeriesSplit
    for train_idx, val_idx in tscv.split(X_seq):
        X_train, X_val = X_seq[train_idx], X_seq[val_idx]
        y_train, y_val = y_seq[train_idx], y_seq[val_idx]
        
        # Definir el modelo LSTM con los hiperparámetros sugeridos
        model = Sequential([
            Input(shape=(72, len(selected_features))),  # Specify the input shape here
            LSTM(lstm_units, activation='relu'),
            Dropout(dropout_rate),
            Dense(1)
        ])
        
        # Compilar el modelo
        model.compile(optimizer='adam', loss='mse')
        
        # Entrenar el modelo
        model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)
        
        # Evaluar el modelo en el conjunto de validación
        y_pred = model.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        mse_scores.append(mse)
    
    # Return the mean MSE across all folds
    return np.mean(mse_scores)

# Ejecutar la optimización con Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:")
print(study.best_params)

# Imprimir el mejor valor de la función objetivo
print("Mejor MSE:")
print(study.best_value)

# Entrenar el modelo final con los mejores hiperparámetros
best_lstm_units = study.best_params['lstm_units']
best_dropout_rate = study.best_params['dropout_rate']

final_model = Sequential([
    LSTM(best_lstm_units, input_shape=(72, len(selected_features)), activation='relu'),
    Dropout(best_dropout_rate),
    Dense(1)
])

final_model.compile(optimizer='adam', loss='mse')

# Reentrenar con todos los datos
final_model.fit(X_seq, y_seq, epochs=10, batch_size=32)

# Validación cruzada con series temporales (usando el modelo final)
tscv = TimeSeriesSplit(n_splits=3)
results = {'LSTM_Optuna': {'mse': [], 'mae': [], 'r2': []}}

for train_idx, test_idx in tscv.split(X_seq):
    X_train, X_test = X_seq[train_idx], X_seq[test_idx]
    y_train, y_test = y_seq[train_idx], y_seq[test_idx]
    
    # Usar el modelo entrenado con Optuna
    y_pred = final_model.predict(X_test)
    
    # Calcular métricas
    results['LSTM_Optuna']['mse'].append(mean_squared_error(y_test, y_pred))
    results['LSTM_Optuna']['mae'].append(mean_absolute_error(y_test, y_pred))
    results['LSTM_Optuna']['r2'].append(r2_score(y_test, y_pred))

[I 2025-03-22 20:41:59,591] A new study created in memory with name: no-name-aa3f668e-9042-47d9-b059-68b47290e3ec
2025-03-22 20:42:03.460885: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 149185152 exceeds 10% of free system memory.
2025-03-22 20:42:03.692188: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 149185152 exceeds 10% of free system memory.
2025-03-22 20:44:18.809821: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 149174784 exceeds 10% of free system memory.


450/450 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
450/450 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


: 

In [ ]:
# Mostrar resultados
for model_name in results:
    print(f"\nResultados para {model_name}:")
    print(f"MSE medio: {np.mean(results[model_name]['mse']):.4f} ± {np.std(results[model_name]['mse']):.4f}")
    print(f"MAE medio: {np.mean(results[model_name]['mae']):.4f} ± {np.std(results[model_name]['mae']):.4f}")
    print(f"R2 medio: {np.mean(results[model_name]['r2']):.4f} ± {np.std(results[model_name]['r2']):.4f}")

# Visualizar comparación de modelos
plt.figure(figsize=(10, 6))
for model_name in results:
    plt.plot(results[model_name]['mae'], label=model_name)
plt.xlabel('Fold')
plt.ylabel('MAE')
plt.title('Comparación de MAE por modelo')
plt.legend()
plt.show()